# INIT

In [1]:
from modules.node import VideoReader, Tracker, TrackerPayload, Render
from modules.FaceMesh import FaceMeshHandler
from queue import Queue
from multiprocessing import Queue as mpQueue


from omegaconf import OmegaConf

video_reader_conf = OmegaConf.load('configs/video_reader.json')
tracker_conf = OmegaConf.load('configs/tracker.json')
render_conf = OmegaConf.load('configs/render.json')
face_mesh_conf = OmegaConf.load('configs/face_mesh.json')

frame_queue = Queue()
render_queue = mpQueue()

video_reader = VideoReader(video_reader_conf, frame_queue)
face_mesh_handler = FaceMeshHandler(face_mesh_conf)
tracker = Tracker(tracker_conf, face_mesh_handler, frame_queue, render_queue)
render = Render(render_conf, render_queue)